In [1]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack',
        'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9,
         'Ten':10, 'Jack':10, 'Queen': 10, 'King':10, 'Ace':11}

playing = True

In [2]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.rank + " of " + self.suit

In [3]:
class Deck:
    
    def __init__(self):
        # Start with an empty list
        self.deck = []
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit, rank))
    
    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n' + card.__str__()
        return "The deck has: " + deck_comp
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def deal(self):
        single_card = self.deck.pop()
        return single_card

In [4]:
class Hand:
    def __init__(self):
        self.cards = []   # Start with an empty list
        self.value = 0    # Start with zero value
        self.aces = 0     # Add an attribute to keep track of aces
        
    def add_card(self, card):
        # card passed in
        # from Deck.deal() --> single Card(suit,rank)
        self.cards.append(card)
        self.value += values[card.rank]
        
        # Track aces
        if card.rank == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self):
        
        # If total value > 21 and still have an ace
        # than change ace to be a 1 instead of an 11
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [5]:
class Chips:
    
    def __init__(self):
        self.total = 100
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

In [6]:
test_deck = Deck()
test_deck.shuffle()

# PLAYER
test_player = Hand()
# Deal 1 card from the deck Card(suit,rank)
pulled_card = test_deck.deal()
print(pulled_card)
test_player.add_card(pulled_card)
print(test_player.value)

Nine of Hearts
9


In [7]:
test_player.add_card(test_deck.deal())

In [8]:
test_player.value

11

In [9]:
def take_bet(chips):
    
    while True:
        
        try:
            chips.bet = int(input("How many chips would you like to bet? "))
        except:
            print("Sorry, please provide an integer.")
        else:
            if chips.bet > chips.total:
                print("Sorry, you do not have enough chips. You have: {}".format(chips.total))
            else:
                break

In [10]:
def hit(deck,hand):
    
    single_card = deck.deal()
    hand.add_card(single_card)
    hand.adjust_for_ace()

In [11]:
def hit_or_stand(deck,hand):
    global playing
    
    while True:
        x = input("Hit or Stand? Enter h or s: ")
        
        if x[0].lower() == 'h':
            hit(deck,hand)
        elif x[0].lower() == 's':
            print("Player Stands. Dealer's Turn.")
            playing = False
        else:
            print("Sorry, please enter h or s only!")
            continue
        break

In [12]:
def show_some(player,dealer):
    
    # dealer.cards[1]
    
    # Show only one of the dealer's cards
    print("\n Dealer's Hand: ")
    print("First card hidden")
    print(dealer.cards[1]) # Show the second card (hide the first one)
    
    # Shows all (2 cards) of player's hand
    print("\n Player's hand:")
    for card in player.cards:
        print(card)

def show_all(player,dealer):
    
    # Show all dealer's cards
    print("\n Dealer's hand:")
    for card in dealer.cards:
        print(card)
    
    # print("\n Dealer's hand:",*dealer.cards,sep'\n')
        
    # Calculate and display value
    print(f"Value of Dealer's hand is: {dealer.value}")
    
    # Show all player's cards
    print("\n Player's hand:")
    for card in player.cards:
        print(card)
        
    # Calculate and display value
    print(f"Value of Player's hand is: {player.value}")

In [13]:
def player_busts(player,dealer,chips):
    print("BUST PLAYER!")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("PLAYER WINS!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("PLAYER WINS! DEALER BUSTED!")
    chips.lose_bet()

def dealer_wins(player,dealer,chips):
    print("DEALER WINS!")
    chips.lose_bet()

def push(player,dealer):
    print("Dealer and Player tie! PUSH!")

In [14]:
while True:
    # Print an opening statement
    print("WELCOME TO BLACKJACK")
    
    # Create & shuffle the deck, deal two cards to each player
    deck = Deck()
    deck.shuffle()
    
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Set up the Player's chips
    player_chips = Chips()
    
    # Prompt the player for their bet
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player_hand,dealer_hand)
    
    while playing: # variable from hit_or_stand function
        
        # Prompt for player to Hit or Stand
        hit_or_stand(deck,player_hand)
        
        # Show cards (but keep one dealer card hidden)
        show_some(player_hand,dealer_hand)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            break
    
    # If the player hasn't busted, play Dealer's hand until Dealer reaches 17
    if player_hand.value <= 21:
        
        while dealer_hand.value < 17:
            hit(deck,dealer_hand)
    
        # Show all cards
        show_all(player_hand, dealer_hand)
        
        # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand,dealer_hand,player_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand,dealer_hand,player_chips)
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand,dealer_hand,player_chips)
        else:
            push(player_hand,dealer_hand)
    
    # Inform Player of their chips total
    print("\n Player's total chips are: {}".format(player_chips.total))
    
    # Ask to play again
    new_game = input("Would you like to play another hand? y/n ")
    
    if new_game[0].lower() == 'y':
        playing = True
        continue
    else:
        print("Thank you for playing!")
        break

WELCOME TO BLACKJACK
How many chips would you like to bet? 300
Sorry, you do not have enough chips. You have: 100
How many chips would you like to bet? 50

 Dealer's Hand: 
First card hidden
Seven of Spades

 Player's hand:
Jack of Hearts
Ace of Diamonds
Hit or Stand? Enter h or s: s
Player Stands. Dealer's Turn.

 Dealer's Hand: 
First card hidden
Seven of Spades

 Player's hand:
Jack of Hearts
Ace of Diamonds

 Dealer's hand:
Three of Diamonds
Seven of Spades
Ten of Diamonds
Value of Dealer's hand is: 20

 Player's hand:
Jack of Hearts
Ace of Diamonds
Value of Player's hand is: 21
PLAYER WINS!

 Player's total chips are: 150
Would you like to play another hand? y/n n
Thank you for playing!
